In [1]:
import numpy as np
import pandas as pd 
import gurobipy as gp
import os 
from gurobipy import Model, GRB, quicksum
import nbformat
import nbimporter
import osmnx as ox
import networkx as nx
import pickle
import glob
from scipy.sparse import lil_matrix
import itertools

In [ ]:
sds

### Preparation

Import the trip csv data

In [2]:
df = pd.read_csv('2019-6_after_filtering.csv')
df

,trip_distance,origin_ID,destination_ID,fare_amount,total_amount,starting_time,ending_time,duration_time,day_type,hour,time_category
0,0.00,262,263,2.5,6.30,2019-06-01 00:06:31,2019-06-01 00:06:52,0.350000,weekend,0,weekend
1,1.70,113,148,9.5,15.95,2019-06-01 00:03:25,2019-06-01 00:15:42,12.283333,weekend,0,weekend
2,1.60,79,125,9.5,14.30,2019-06-01 00:28:31,2019-06-01 00:39:23,10.866667,weekend,0,weekend
3,0.60,211,148,4.5,8.30,2019-06-01 00:46:46,2019-06-01 00:50:55,4.150000,weekend,0,weekend
4,1.20,79,249,7.5,12.30,2019-06-01 00:54:49,2019-06-01 01:02:57,8.133333,weekend,0,weekend
...,...,...,...,...,...,...,...,...,...,...,...
5458387,0.90,68,158,11.0,16.80,2019-06-30 23:23:03,2019-06-30 23:39:48,16.750000,weekend,23,weekend
5458388,0.50,246,90,6.0,9.80,2019-06-30 23:50:22,2019-06-30 23:57:01,6.650000,weekend,23,weekend
5458389,0.20,90,186,3.5,8.75,2019-06-30 23:58:32,2019-07-01 00:00:42,2.166667,weekend,23,weekend
5458390,1.38,140,163,7.5,13.56,2019-06-30 23:23:10,2019-06-30 23:30:45,7.583333,weekend,23,weekend


Scenarios definition

note that here we divide demand level into three levels: low demand ( lower than 5 trips), medium demand(6 trips to 70 trips), and high demand (larger than 70 trips). 

Now we can have 24 * 3939 * 3 = 283608 scenarios, and **subsequent reductions** may occur. 

In [3]:
demand_df = df.groupby(['origin_ID', 'destination_ID', 'hour']).size().reset_index(name='count')
def demand_category(x):
    if x < 6:
        return 'low demand'
    elif 6 <= x <= 70:
        return 'medium demand'
    else:
        return 'high demand'
        

demand_df['demand_level'] = demand_df['count'].apply(demand_category)
demand_df = demand_df.drop(['count'], axis=1)


existing_trips_df = df.groupby(['origin_ID', 'destination_ID']).size().reset_index(name='count')
# Create a new list of scenario
scenarios = []

for hour in range(24):
    for index, row in existing_trips_df.iterrows():
        scenarios.append((hour, row['origin_ID'], row['destination_ID']))

Import the speed and graph data

In [4]:
# Import Manhattan network and change node labels to integers
G = ox.graph_from_place('Manhattan, New York, USA', network_type='drive')

# If a node cannot access at least 10% of other nodes, delete it (isolated points are not considered)
remove_list = []
num_nodes = len(G.nodes)
for node in G.nodes:  
    reach = len(nx.descendants(G, node))
    if reach < num_nodes / 10:
        remove_list.append(node)

for node in remove_list:
    G.remove_node(node)

# The node labels of the graph are converted to integers for easier handling and reference, 
G = nx.convert_node_labels_to_integers(G, label_attribute='old_node_ID')
G = ox.add_edge_speeds(G)
speed_df_path = r"C:\Users\yanzh\Desktop\Code\archive\nyc_avg_speeds_2019-06.csv"
speed_df = pd.read_csv(speed_df_path)
speed_df = speed_df[['osm_way_id', 'hour', 'speed']]
G = ox.add_edge_travel_times(G, precision=1)


# Load the graphs data with speed in the file
def load_graph_from_pkl(file_path):

    with open(file_path, "rb") as f:
        graph = pickle.load(f)
    return graph

# The graphs holds a list of graph objects for each hour, here is 24 hour
graphs = load_graph_from_pkl("graphs_list.pkl")

# Go through the graph in each hour
for hour in range(24):
    for u, v, data in graphs[hour].edges(data=True):
        travel_time_key = f'travel_time_hour_{hour}'

        if travel_time_key not in data:
           freeflow_travel_time = G[u][v][0].get('travel_time', None)
           if freeflow_travel_time is not None:
               graphs[hour][u][v][0][travel_time_key] = freeflow_travel_time

C:\Users\yanzh\AppData\Local\Temp\ipykernel_15220\1008873551.py:21: FutureWarning: The `precision` parameter is deprecated and will be removed in the v2.0.0 release.
  G = ox.add_edge_travel_times(G, precision=1)


In [ ]:
# def load_graph_from_pkl(file_path):
#     with open(file_path, "rb") as f:
#         graphs = pickle.load(f)
#     return graphs

# def process_graphs(graphs):
#     # Verify if each graph is an instance of MultiGraph
#     for hour, graph in enumerate(graphs):
#         if isinstance(graph, nx.MultiGraph):
#             update_graph_edges(graph, hour)
#         else:
#             print(f"Graph for hour {hour} is not a MultiGraph.")

# def update_graph_edges(graph, hour):
#     travel_time_key = f'travel_time_hour_{hour}'
#     for u, v, key, data in graph.edges(keys=True, data=True):
#         if travel_time_key not in data:
#             # Safely access the travel time in a multigraph structure
#             freeflow_travel_time = graph[u][v][key].get('travel_time', None)
#             if freeflow_travel_time is not None:
#                 graph[u][v][key][travel_time_key] = freeflow_travel_time

# # Load and process the graphs
# graphs = load_graph_from_pkl("/content/drive/MyDrive/graphs_list.pkl")
# process_graphs(graphs)


In [5]:
# Define the travel time function using Dijkstra algorithm
def travel_time_func(G_hour, point1, point2, hour):
    # Define the weight key for the specific hour
    weight_key = f'travel_time_hour_{hour}'

    # Use Dijkstra's algorithm to find the shortest path length and path
    # This function returns both the length of the path and the actual path as a list of nodes
    travel_time, path = nx.single_source_dijkstra(G_hour, source=point1, target=point2, weight=weight_key)

    # Round the travel time to 2 decimal places
    travel_time = round(travel_time, 4)

    return travel_time, path

### Constants

In [6]:
# number of scenarios
S = len(scenarios)

# number of trips
K = len(df)

# number of poential charging staion locations
n_locations = 65 

# number of cars 
n_cars = 50

# number of scenarios 
n_scenarios = 5 

# fixed cost of each charging station, here we assmue 150000 dollars, but here because we only use 10000 cars, so we need reduce the cost
station_cost = 150000

# Purchasing cost of each car
car_cost = 15000 # Here we use 15000 dollars

# Income of each accepted trip
income_per_car = 50 # assume here is 50 dollars

# Capacity of each charging station, i.e. number of charging slots can be built at each charging station
capacity = 5

# p_s is a dictionary containing the profit weights for each scenario
# Here assume equal probability for each scenario
p_s = 1 / len(scenarios)
# Assuming i_k is a dictionary or a function that provides income for each trip k in scenario s ?

# Time periods 
T = 24

# Potential location of charging station, and then number them from 0 to 64
I = n_locations

# Cars
H = n_cars


### Time-expanded location graphs  G=(V, A)

Note that there are Duplicate travel arcs which means some cars have same trips at the same scenario, location and time

In [25]:
# Read the csv file of coordinatate of charging station
df_charging_station_location = pd.read_csv('coordinates of charging station.csv')

# First is V, set of nodes in the graph
root = 'root'
sink = 'sink'

V0 = [(s, i, t) for s in S for i in I for t in T]
V = [root] + [(s, i, t) for s in S for i in I for t in T] + [sink] # Here V is described as a set of tuples, with each tuple contains two integers
                                                     #Plus the root and sink  

# Waiting arcs
waiting_arcs = [(s, (i, t), (i, t+1)) for s in S for i in I for t in list(range(23))]

# The traveling arcs 
location_id_to_index = df_charging_station_location.set_index('Location_id')['index'].to_dict()
travel_arcs = []

for s in S:
    for k in range(trips_each_scenario):

        # Select the specific row
        row = Ks[s].iloc[k]
        
        
        # change the starting point id to index (from 0 to 65)
        starting_point_id = row['origin_ID']
        if starting_point_id in location_id_to_index:
            starting_point = location_id_to_index[starting_point_id]

        # Convert time to hour    
        starting_time = row['starting_time'].hour
        
        # Change the destination point id to index (from 0 to 65)
        ending_point_id = row['destination_ID']
        if ending_point_id in location_id_to_index:
            ending_point = location_id_to_index[ending_point_id]
        
        # Convert time to hour    
        ending_time = row['ending_time'].hour

        arcs = (s, k, (starting_point, starting_time), (ending_point, ending_time))
        travel_arcs.append(arcs)


# Initial allocation arcs
initial_allocation_arcs = [(s, root, (i, 0)) for s in S for i in I ]

final_collection_arcs = [(s, (i, 23), sink) for s in S for i in I ]

# Define arcs excluding travel arcs
arcs_exclduing_travel_arcs = waiting_arcs + initial_allocation_arcs + final_collection_arcs

all_arcs = initial_allocation_arcs + waiting_arcs + travel_arcs + final_collection_arcs

TypeError: 'int' object is not iterable

### Decision variable

First stage variable (Strategical layer)
$$
y_i=1 \ \text{if the charging station is built at $i$ point}, \forall i \in I
$$
$$
L_i= \ \text{initial number of EVs at station i}, \forall i \in I
$$

Second stage variable(Operational layer)
$$
x_k=1 \ \text{if the $k$ trip is accepted}, k \in K^s, \forall s\in S
$$
$$
x_k^h = 1 \ \text{if and only if an accepted trip $k$ of scenario $s$ will be realized by purchased car $h$}
$$
$$
f_a^h = 1 \ \text{if the car $h$ travels from station $i$ at time $t$ to station $j$ at time $t^{\prime}$}
$$

In [7]:
# Initialize the model
m = Model('CSLP')

# m.setParam('NodefileStart', 0.5)

# First stage decision variable
y_i = m.addVars(range(n_locations), vtype=GRB.BINARY, name='build_variable')
L_i = m.addVars(range(n_locations), vtype=GRB.INTEGER, name='purchased_car', lb=0, ub=10)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-08


In [16]:
scenarios_df = pd.DataFrame(scenarios, columns=['hour', 'origin_ID', 'destination_ID'])

# Merge 'df' with 'scenarios_df' to find the corresponding scenario for each trip
merged_df = df.merge(scenarios_df, on=['hour', 'origin_ID', 'destination_ID'], how='inner')

# The row indices for the sparse matrix are the index of the trips
rows = merged_df.index.to_numpy()

In [20]:
cols = merged_df['index'].to_numpy()

cols

KeyError: 'index'

In [ ]:
# Create a matrix of zeros with the dimensions of (len(df), len(scenarios))
x_k_matrix = np.zeros((len(df), len(scenarios)), dtype=int)

In [ ]:
# Second stage decision variale
indices = [(k, s) for k in K for s in S if x_k[k, s] == 1]
x_vars = m.addVars(indices, vtype=GRB.BINARY, name='whether trip is accepted')


In [ ]:
# Only creat the (k, s, h) only for valid (k, s) pairs
valid_ks = [(k, s) for k, s in indices]
valid_ksh = [(k, s, h) for k, s in valid_ks for h in H]

x_hk = m.addVars(valid_ksh, vtype=GRB.BINARY, name='trip_realized_by_car')


In [ ]:
# flow variable in the second stage
f_ha = m.addVars(range(len(all_arcs)), vtype=GRB.BINARY, name='flow') 

### Objective function

$$Max\ \sum_{s\in S}p_s\sum_{k\in K^s}{i_kx_k\ -\ \sum_{i\in I}{f_iy_i-c\sum_{i\in I}L_i}}$$

In [131]:
# Objective function: maximize profit
m.setObjective(
    quicksum(p_s[s] * income_per_car * quicksum(x_k[s, k] for k in range(trips_each_scenario)) for s in S) -
    station_cost * quicksum(y_i[i] for i in range(n_locations)) -
    car_cost * quicksum(L_i[i] for i in range(n_locations)),
    GRB.MAXIMIZE
)

Some mandatory requirments

In [132]:
# At least 5 stations should be built
m.addConstr(quicksum(y_i[i] for i in I) >= 5, name="at_least_five_stations")

# Total number of cars should be fixed as 50 cars
m.addConstr(quicksum(L_i[i] for i in I) == 50, name='total_number_of_cars')

# If the station is built in i, L_[i] should more than 0.
for i in I:
    m.addConstr(L_i[i] >= 1 - 1000 * (1 - y_i[i]), name=f'min_cars_if_station_{i}')

# Initil capacity limitation
for i in I:
    m.addConstr(L_i[i] <= capacity, name=f'initial capacity limitation_{i}') 
    

### Constarints

#### Constraint 1
$$\sum_{i\in I}{f_i y_i - c\sum_{i\in I} L_i} \leq W$$

This is the budget constraint, $W$ is the limited budget for all costs.

In [101]:
# Budget constraints
budget = 10000000
m.addConstr(station_cost * quicksum(y_i[i] for i in range(n_locations)) + 
            car_cost * quicksum(L_i[i] for i in range(n_locations)) <= budget, 
            name='budge_constraint')

<gurobi.Constr *Awaiting Model Update*>

### Constraint 2

$$0< t^s_k x_k \leq T^{max}\qquad\forall s \in S, k \in K^s$$

The travel time of the $k$ trip should not exceed the maximum travel time of the car due to battery limitation.

In [102]:
# Battery limitation

# The maximum operational time is 60 minutes
T_max = 60 

# Create a mapping from Location id to networkx point, here I create a dictionary
location_id_to_networkx_point = df_charging_station_location.set_index('Location_id')['networkx_point'].to_dict()

for s in S:
    for k in range(K):

        # Get the orgin_id value right now
        origin_id = Ks[s]['origin_ID'].iloc[k]
        

        if origin_id in location_id_to_networkx_point:
            origin = location_id_to_networkx_point[origin_id]
        
        # Get the destination_id value right now
        destination_id = Ks[s]['destination_ID'].iloc[k]
        
        if destination_id in location_id_to_networkx_point:
            destination = location_id_to_networkx_point[destination_id]

        hour = Ks[s]['starting_time'].dt.hour.iloc[k]
        G_hour = graphs[hour]

        #Calculate travel time for this trip
        travel_time, _ = travel_time_func(G_hour, origin, destination, hour)

        # # Check for NaN or Inf values
        # if np.isnan(travel_time) or np.isinf(travel_time):
        #     print(f"Invalid travel time detected: Scenario {s}, Trip {k}, Origin {origin}, Destination {destination}, Hour {hour}, Travel Time {travel_time}")
        #     continue

        # Only apply the constraint if the trip is accepted
        m.addConstr(travel_time * x_k[s, k] <= T_max, name=f"Battery limitation_s{s}_k{k}")


### Constraints 3

$$\sum_{h=1}^H x_k^h = x_k, \qquad\forall s \in S, k \in K^s $$

It ensures that exactly one car is assigned to each accepted trip.

In [103]:
for s in S:
    for k in range(trips_each_scenario):
        m.addConstr(quicksum(x_hk[s, h, k] for h in H) == x_k[s, k], name=f"one_car_per_accepted_trip_s{s}_k{k}")
            

### Constrain 4

$$\sum_{h=1}^H \sum_{a \in \delta^{+}\left(i_t\right) \cap\left(A_W^s \cup A_C^s\right)} f_a^h \leq C_i y_i \qquad \forall s \in S, \quad\forall i_t \in V^s \backslash\left\{r^s, s^s\right\}$$

It ensures that the quantity of vehicles concurrently parked at station $i$ does not surpass the available number of charging slots at said station.
Observe that final collection arcs need to be considered on the left-hand side to ensure that the capacity constraints are also met at the end of the planning period.

In [89]:
# Capacity constraints

for s in S:
    for i in I:
        # Directly handling the final collection arcs to 'sink' 
        final_arc_key = (s, (i, 23), 'sink')
        m.addConstr(quicksum(f_ha[s, h, final_arc_key] for h in H if final_arc_key in f_ha) <= capacity * y_i[i], name=f'final_capacity_s{s}_i{i}')
        
        for t in T:         
            # Filter and append arcs relevant to the current (s, i, t)
            outgoing_waiting_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and src == (i, t)]
        
            # Add constraint            
            m.addConstr(quicksum(f_ha[s, h, arc] for h in H for arc in outgoing_waiting_arcs if arc in f_ha) <= capacity * y_i[i])


### Constraints 5

$$    f^h[\delta^{-}\left(i_t\right)] \leq y_i \quad \forall h \in\{1,2, \ldots, H\}, \quad \forall s \in S, \quad \forall i_t \in V^s \backslash\left\{r^s, s^s\right\}$$

It ensures that car can only enter the built station. It includes waiting arcs (cars only car wait at the built station), and traveling arcs (cars can only park at the built station)

In [162]:
# Constraint 5
for s in S:
    for i in I:
        for t in T:
            
            # Given s, i, t, a specific waiting arc can be identified
            incoming_waiting_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and dst == (i, t)]

            # Add the constraint
            m.addConstr(quicksum(f_ha[s, h, arc] for h in H for arc in incoming_waiting_arcs if arc in f_ha) <= y_i[i])

            # Next is travel arc

            incoming_travel_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s and dst == (i, t)]

            # Add the constraint
            m.addConstr(quicksum(f_ha[s, h, arc] for h in H for arc in incoming_travel_arcs if arc in f_ha) <= y_i[i])

### Constraint 6

$$f^h\left[\delta^{-}\left(i_t\right)\right]=f^h\left[\delta^{+}\left(i_t\right)\right] \quad \forall h \in\{1,2, \ldots, H\}, \quad\forall s \in S, \quad\forall i_t \in V^s \backslash\left\{r^s, s^s\right\}$$

Flow conservation ensures that the route of each car must correspond to a path through the time-expanded location graph for each scenario

In [163]:
for s in S:
    for i in I:
        for t in T:
            
            # Given s, i, t, a specific incoming  arc can be identified
            incoming_waiting_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and dst == (i, t)]
            incoming_travel_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s and dst == (i, t)]
            incoming_arcs = incoming_travel_arcs + incoming_waiting_arcs

            # Given s, i, t, a specific outgoing  arc can be identified
            outgoing_waiting_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and src == (i, t)]
            outgoing_travel_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s and src == (i, t)]
            outgoing_arcs = outgoing_travel_arcs + outgoing_waiting_arcs

            for h in H:
                # Add constraints
                m.addConstr(
                    quicksum(f_ha[s, h, arc] for arc in incoming_arcs if arc in f_ha) ==
                    quicksum(f_ha[s, h, arc] for arc in outgoing_arcs if arc in f_ha),
                    name=f"flow_conservation_s{s}_i{i}_t{t}_h{h}")

            

### Constraint 7 

$$\sum_{a \in A_{T}^s(k)} f_a^h=x_k^h \quad \forall h \in\{1,2, \ldots, H\}, \quad \forall s \in S, \quad \forall k \in K^s$$

This equation illustrates all action of one car.

In [167]:
# Precompute travel arcs for each scenario and trip
travel_arcs_per_scenario_trip = {
    (s, k): [arc for arc in travel_arcs if arc[0] == s and arc[1] == k]
    for s in S
    for k in range(trips_each_scenario)
}

# Now add constraints using the precomputed arcs
for s in S:
    for k in range(trips_each_scenario):
        for h in H:
            relevant_arcs = travel_arcs_per_scenario_trip[(s, k)]
            sum_f_ha_for_travel_arcs = quicksum(f_ha[arc] for arc in relevant_arcs)
            m.addConstr(sum_f_ha_for_travel_arcs == x_hk[s, h, k], name=f'all_action_one_car_s{s}_h{h}_k{k}')



KeyError: (0, 0, (18, 0), (20, 0))

In [149]:
# for s in S:
#     for k in range(trips_each_scenario):
#         for h in H:
#             # Calculate the sum of f_ha for all travel arcs for a specific scenario s, car h and trip k
#             sum_f_ha_for_travel_arcs = quicksum(f_ha[s, k, h, arc] for arc in travel_arcs if arc in f_ha)
#             m.addConstr(sum_f_ha_for_travel_arcs == x_hk[s, h, k], name=f'all_action_one_car_s{s}_i{i}_t{t}_h{h}')

### Constraint 8

$$ f_a^h \leq f_{a^{\prime}}^h \\

\forall h \in\{1,2, \ldots, H\}, \forall s \in S, \forall k \in K^s, \\
\forall a=\left(i_{s_k}, j_{e_k}\right) \in A_{\mathrm{T}}^s(k), \\
\forall a^{\prime}=\left(j_t, j_{t^{\prime}}\right) \in A_{W}^s, \\
t = e_k, t^{\prime} = e_k+\left\lceil\frac{b_k}{\rho}\right\rceil $$

This equation force each car must fully charge the battery after completing the service.

In [148]:
# Assume fully charging time is 60 mins
fully_charging_time = 1

for s in S:
    for k in range(trips_each_scenario):
        
        # Define AI_arcs considering all travel arcs for services s.
        AT_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s ] 
        if AT_arcs:
            for h in H:
                
                at_arc = AT_arcs[k]
                st = at_arc[2][1]
                et = at_arc[3][1]

                t = et
                t_prime = t + fully_charging_time

                # Finding the corresponding AW_arcs with the updated time t_prime
                AW_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and src[1] == t and dst[1] == t_prime]
                    
                    
                # Now, add the constraint for each matching arc, ensuring we use the unique arc identifiers from all_unique_arcs
                for aw_arc in AW_arcs:
                    if (s, h, at_arc) in f_ha and (s, h, aw_arc) in f_ha:
                        m.addConstr(f_ha[s, h, at_arc] <= f_ha[s, h, aw_arc], f"charging_constraint_{s}_{k}_{h}_{at_arc}_{aw_arc}")
                        print(f'finding the corresponding matching arc_s{s}_h{h}_k{k}')


# 4. Solve the model

In [133]:
# Solve the model
m.optimize()

# Directly check the optimization status
if m.Status == GRB.OPTIMAL:
    print("Optimization was successful. Printing results.")
    for i in range(n_locations):  # Ensure n_locations is correctly set
        if y_i[i].X > 0.5:
            print(f"Build a charging station at location {i} with {L_i[i].X} cars.")
else:
    print(f"Optimization issue with status code: {m.Status}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 9 7945HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 132 rows, 528255 columns and 325 nonzeros
Model fingerprint: 0x6c834621
Variable types: 0 continuous, 528255 integer (528190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+01, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [5e+00, 1e+03]
Found heuristic solution: objective 185000.00000
Presolve removed 65 rows and 528125 columns
Presolve time: 0.03s
Presolved: 67 rows, 130 columns, 260 nonzeros
Variable types: 0 continuous, 130 integer (65 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node